In [5]:
import pandas as pd

#load dataset
df = pd.read_csv(r'/Users/bhoomikaharkhani/Downloads/noshowappointments.csv')

#rename columns for consistency
df = df.rename(columns = {
    'PatientId': 'patient_id',
    'AppointmentID': 'appointment_id',
    'Gender': 'gender',
    'ScheduledDay': 'scheduled_day',
    'AppointmentDay': 'appointment_day',
    'Age': 'age',
    'Neighbourhood': 'neighbourhood',
    'Scholarship': 'scholarship',
    'Hipertension': 'hypertension',
    'Diabetes': 'diabetes',
    'Alcoholism': 'alcoholism',
    'Handcap': 'handicap',
    'SMS_received': 'sms_received',
    'No-show': 'no_show'
})

#convert no-show to 0/1
df['no_show'] = df['no_show'].map({'No': 0, 'Yes': 1})

#convert datetime columns
df['scheduled_day'] = pd.to_datetime(df['scheduled_day']).dt.tz_localize(None)
df['appointment_day'] = pd.to_datetime(df['appointment_day']).dt.date

#clean binary columns
binary_fields = ['scholarship','hypertension','diabetes','alcoholism','handicap','sms_received','no_show']
df[binary_fields] = df[binary_fields].astype(int)

#drop duplicate on appointment_id
df = df.drop_duplicates(subset = ['appointment_id'])

#create dim_patient
dim_patient = df[['patient_id', 'gender', 'age', 'scholarship', 'hypertension', 'diabetes', 'alcoholism', 'handicap']].drop_duplicates()

#create dim_location
dim_location = df[['neighbourhood']].drop_duplicates().reset_index(drop=True)
dim_location['location_id'] = dim_location.index + 1

#merge to add locatoin id
df = df.merge(dim_location, on = 'neighbourhood', how='left')

#create face appointment
fact_appointment = df[['appointment_id', 'patient_id', 'location_id', 'scheduled_day', 'appointment_day', 'sms_received', 'no_show']]

#save files
dim_patient.to_csv('dim_patient.csv', index=False, encoding="utf-8")
dim_location.to_csv('dim_location.csv', index=False, encoding="utf-8")
fact_appointment.to_csv('fact_appointment.csv', index=False, encoding="utf-8")

print("✅ Cleaned CSVs saved!")

✅ Cleaned CSVs saved!


In [6]:
import pandas as pd

# Load the CSV
df = pd.read_csv("dim_patient_ascii_clean.csv")

# Drop duplicate patient_id values, keeping the first occurrence
df_unique = df.drop_duplicates(subset="patient_id", keep="first")

# Save the cleaned file
df_unique.to_csv("dim_patient_cleaned.csv", index=False)
